In [ ]:
!git clone https://github.com/facebookresearch/audiocraft.git
%cd audiocraft
# !pip install torch==2.2.1
!pip install -e .
!pip install dora-search
!pip install numba

# data preprocessing etc

This section includes a tool to slice the audio into 30s chunks, resample to 44100hz, and normalize. it also includes a WIP autolabeller based on essentia (https://essentia.upf.edu/models.html, https://colab.research.google.com/drive/1tFInmCYK2uX-PajYemERvtSojkT0vrhF)

Tags parsed by essentia:
- genre
- mood/theme
- instrumentation
- key, bpm (these are actually from librosa)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# mount google drive to access dataset
# from google.colab import drive
# drive.mount('/content/drive')

dataset_path = "/home/murali/data/samples/train_musicgen_edm_uncond/train/outputs/"

In [ ]:
# split and resample
# don't run this if your audio is already sliced and resampled

import os
from pydub import AudioSegment

os.makedirs(os.path.join(dataset_path, "original"), exist_ok=True)

for filename in os.listdir(dataset_path):
    if filename.endswith(('.mp3', '.wav', '.flac')):

        # move original file out of the way
        # os.rename(filename, f"original/{filename}")
        # audio = AudioSegment.from_file(f"original/{filename}")

         # Move original file to the 'original' directory
        original_file_path = os.path.join(dataset_path, filename)
        new_file_path = os.path.join(dataset_path, "original", filename)
        os.rename(original_file_path, new_file_path)


        # audio = AudioSegment.from_file(f"original/{filename}")
        audio = AudioSegment.from_file(new_file_path)



        # resample
        audio = audio.set_frame_rate(44100)

        # split into 30-second chunks
        for i in range(0, len(audio), 30000):
            chunk = audio[i:i+30000]
            chunk.export(f"{new_file_path[:-4]}_chunk{i//1000}.wav", format="wav")


In [ ]:
# install essentia and requirements

# this line is needed for the tensorflow modules to install properly!!
!sudo apt-get install build-essential libeigen3-dev libyaml-dev libfftw3-dev libtag1-dev libchromaprint-dev

!pip install -U essentia-tensorflow

[sudo] password for murali: 


In [ ]:
# download some essentia model weights

!curl https://essentia.upf.edu/models/classification-heads/genre_discogs400/genre_discogs400-discogs-effnet-1.pb --output genre_discogs400-discogs-effnet-1.pb
!curl https://essentia.upf.edu/models/feature-extractors/discogs-effnet/discogs-effnet-bs64-1.pb --output discogs-effnet-bs64-1.pb
!curl https://essentia.upf.edu/models/classification-heads/mtg_jamendo_moodtheme/mtg_jamendo_moodtheme-discogs-effnet-1.pb --output mtg_jamendo_moodtheme-discogs-effnet-1.pb
!curl https://essentia.upf.edu/models/classification-heads/mtg_jamendo_instrument/mtg_jamendo_instrument-discogs-effnet-1.pb --output mtg_jamendo_instrument-discogs-effnet-1.pb

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2009k  100 2009k    0     0   155k      0  0:00:12  0:00:12 --:--:--  319k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.5M  100 17.5M    0     0   516k      0  0:00:34  0:00:34 --:--:--  668k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2675k  100 2675k    0     0   401k      0  0:00:06  0:00:06 --:--:--  528k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2643k  100 2643k    0     0   517k      0  0:00:05  0:00:05 --:--:--  542k


In [ ]:
# @title metadata (labels) for essentia - LONG CELL DONT OPEN
genre_labels = [
    "Blues---Boogie Woogie",
    "Blues---Chicago Blues",
    "Blues---Country Blues",
    "Blues---Delta Blues",
    "Blues---Electric Blues",
    "Blues---Harmonica Blues",
    "Blues---Jump Blues",
    "Blues---Louisiana Blues",
    "Blues---Modern Electric Blues",
    "Blues---Piano Blues",
    "Blues---Rhythm & Blues",
    "Blues---Texas Blues",
    "Brass & Military---Brass Band",
    "Brass & Military---Marches",
    "Brass & Military---Military",
    "Children's---Educational",
    "Children's---Nursery Rhymes",
    "Children's---Story",
    "Classical---Baroque",
    "Classical---Choral",
    "Classical---Classical",
    "Classical---Contemporary",
    "Classical---Impressionist",
    "Classical---Medieval",
    "Classical---Modern",
    "Classical---Neo-Classical",
    "Classical---Neo-Romantic",
    "Classical---Opera",
    "Classical---Post-Modern",
    "Classical---Renaissance",
    "Classical---Romantic",
    "Electronic---Abstract",
    "Electronic---Acid",
    "Electronic---Acid House",
    "Electronic---Acid Jazz",
    "Electronic---Ambient",
    "Electronic---Bassline",
    "Electronic---Beatdown",
    "Electronic---Berlin-School",
    "Electronic---Big Beat",
    "Electronic---Bleep",
    "Electronic---Breakbeat",
    "Electronic---Breakcore",
    "Electronic---Breaks",
    "Electronic---Broken Beat",
    "Electronic---Chillwave",
    "Electronic---Chiptune",
    "Electronic---Dance-pop",
    "Electronic---Dark Ambient",
    "Electronic---Darkwave",
    "Electronic---Deep House",
    "Electronic---Deep Techno",
    "Electronic---Disco",
    "Electronic---Disco Polo",
    "Electronic---Donk",
    "Electronic---Downtempo",
    "Electronic---Drone",
    "Electronic---Drum n Bass",
    "Electronic---Dub",
    "Electronic---Dub Techno",
    "Electronic---Dubstep",
    "Electronic---Dungeon Synth",
    "Electronic---EBM",
    "Electronic---Electro",
    "Electronic---Electro House",
    "Electronic---Electroclash",
    "Electronic---Euro House",
    "Electronic---Euro-Disco",
    "Electronic---Eurobeat",
    "Electronic---Eurodance",
    "Electronic---Experimental",
    "Electronic---Freestyle",
    "Electronic---Future Jazz",
    "Electronic---Gabber",
    "Electronic---Garage House",
    "Electronic---Ghetto",
    "Electronic---Ghetto House",
    "Electronic---Glitch",
    "Electronic---Goa Trance",
    "Electronic---Grime",
    "Electronic---Halftime",
    "Electronic---Hands Up",
    "Electronic---Happy Hardcore",
    "Electronic---Hard House",
    "Electronic---Hard Techno",
    "Electronic---Hard Trance",
    "Electronic---Hardcore",
    "Electronic---Hardstyle",
    "Electronic---Hi NRG",
    "Electronic---Hip Hop",
    "Electronic---Hip-House",
    "Electronic---House",
    "Electronic---IDM",
    "Electronic---Illbient",
    "Electronic---Industrial",
    "Electronic---Italo House",
    "Electronic---Italo-Disco",
    "Electronic---Italodance",
    "Electronic---Jazzdance",
    "Electronic---Juke",
    "Electronic---Jumpstyle",
    "Electronic---Jungle",
    "Electronic---Latin",
    "Electronic---Leftfield",
    "Electronic---Makina",
    "Electronic---Minimal",
    "Electronic---Minimal Techno",
    "Electronic---Modern Classical",
    "Electronic---Musique Concrète",
    "Electronic---Neofolk",
    "Electronic---New Age",
    "Electronic---New Beat",
    "Electronic---New Wave",
    "Electronic---Noise",
    "Electronic---Nu-Disco",
    "Electronic---Power Electronics",
    "Electronic---Progressive Breaks",
    "Electronic---Progressive House",
    "Electronic---Progressive Trance",
    "Electronic---Psy-Trance",
    "Electronic---Rhythmic Noise",
    "Electronic---Schranz",
    "Electronic---Sound Collage",
    "Electronic---Speed Garage",
    "Electronic---Speedcore",
    "Electronic---Synth-pop",
    "Electronic---Synthwave",
    "Electronic---Tech House",
    "Electronic---Tech Trance",
    "Electronic---Techno",
    "Electronic---Trance",
    "Electronic---Tribal",
    "Electronic---Tribal House",
    "Electronic---Trip Hop",
    "Electronic---Tropical House",
    "Electronic---UK Garage",
    "Electronic---Vaporwave",
    "Folk, World, & Country---African",
    "Folk, World, & Country---Bluegrass",
    "Folk, World, & Country---Cajun",
    "Folk, World, & Country---Canzone Napoletana",
    "Folk, World, & Country---Catalan Music",
    "Folk, World, & Country---Celtic",
    "Folk, World, & Country---Country",
    "Folk, World, & Country---Fado",
    "Folk, World, & Country---Flamenco",
    "Folk, World, & Country---Folk",
    "Folk, World, & Country---Gospel",
    "Folk, World, & Country---Highlife",
    "Folk, World, & Country---Hillbilly",
    "Folk, World, & Country---Hindustani",
    "Folk, World, & Country---Honky Tonk",
    "Folk, World, & Country---Indian Classical",
    "Folk, World, & Country---Laïkó",
    "Folk, World, & Country---Nordic",
    "Folk, World, & Country---Pacific",
    "Folk, World, & Country---Polka",
    "Folk, World, & Country---Raï",
    "Folk, World, & Country---Romani",
    "Folk, World, & Country---Soukous",
    "Folk, World, & Country---Séga",
    "Folk, World, & Country---Volksmusik",
    "Folk, World, & Country---Zouk",
    "Folk, World, & Country---Éntekhno",
    "Funk / Soul---Afrobeat",
    "Funk / Soul---Boogie",
    "Funk / Soul---Contemporary R&B",
    "Funk / Soul---Disco",
    "Funk / Soul---Free Funk",
    "Funk / Soul---Funk",
    "Funk / Soul---Gospel",
    "Funk / Soul---Neo Soul",
    "Funk / Soul---New Jack Swing",
    "Funk / Soul---P.Funk",
    "Funk / Soul---Psychedelic",
    "Funk / Soul---Rhythm & Blues",
    "Funk / Soul---Soul",
    "Funk / Soul---Swingbeat",
    "Funk / Soul---UK Street Soul",
    "Hip Hop---Bass Music",
    "Hip Hop---Boom Bap",
    "Hip Hop---Bounce",
    "Hip Hop---Britcore",
    "Hip Hop---Cloud Rap",
    "Hip Hop---Conscious",
    "Hip Hop---Crunk",
    "Hip Hop---Cut-up/DJ",
    "Hip Hop---DJ Battle Tool",
    "Hip Hop---Electro",
    "Hip Hop---G-Funk",
    "Hip Hop---Gangsta",
    "Hip Hop---Grime",
    "Hip Hop---Hardcore Hip-Hop",
    "Hip Hop---Horrorcore",
    "Hip Hop---Instrumental",
    "Hip Hop---Jazzy Hip-Hop",
    "Hip Hop---Miami Bass",
    "Hip Hop---Pop Rap",
    "Hip Hop---Ragga HipHop",
    "Hip Hop---RnB/Swing",
    "Hip Hop---Screw",
    "Hip Hop---Thug Rap",
    "Hip Hop---Trap",
    "Hip Hop---Trip Hop",
    "Hip Hop---Turntablism",
    "Jazz---Afro-Cuban Jazz",
    "Jazz---Afrobeat",
    "Jazz---Avant-garde Jazz",
    "Jazz---Big Band",
    "Jazz---Bop",
    "Jazz---Bossa Nova",
    "Jazz---Contemporary Jazz",
    "Jazz---Cool Jazz",
    "Jazz---Dixieland",
    "Jazz---Easy Listening",
    "Jazz---Free Improvisation",
    "Jazz---Free Jazz",
    "Jazz---Fusion",
    "Jazz---Gypsy Jazz",
    "Jazz---Hard Bop",
    "Jazz---Jazz-Funk",
    "Jazz---Jazz-Rock",
    "Jazz---Latin Jazz",
    "Jazz---Modal",
    "Jazz---Post Bop",
    "Jazz---Ragtime",
    "Jazz---Smooth Jazz",
    "Jazz---Soul-Jazz",
    "Jazz---Space-Age",
    "Jazz---Swing",
    "Latin---Afro-Cuban",
    "Latin---Baião",
    "Latin---Batucada",
    "Latin---Beguine",
    "Latin---Bolero",
    "Latin---Boogaloo",
    "Latin---Bossanova",
    "Latin---Cha-Cha",
    "Latin---Charanga",
    "Latin---Compas",
    "Latin---Cubano",
    "Latin---Cumbia",
    "Latin---Descarga",
    "Latin---Forró",
    "Latin---Guaguancó",
    "Latin---Guajira",
    "Latin---Guaracha",
    "Latin---MPB",
    "Latin---Mambo",
    "Latin---Mariachi",
    "Latin---Merengue",
    "Latin---Norteño",
    "Latin---Nueva Cancion",
    "Latin---Pachanga",
    "Latin---Porro",
    "Latin---Ranchera",
    "Latin---Reggaeton",
    "Latin---Rumba",
    "Latin---Salsa",
    "Latin---Samba",
    "Latin---Son",
    "Latin---Son Montuno",
    "Latin---Tango",
    "Latin---Tejano",
    "Latin---Vallenato",
    "Non-Music---Audiobook",
    "Non-Music---Comedy",
    "Non-Music---Dialogue",
    "Non-Music---Education",
    "Non-Music---Field Recording",
    "Non-Music---Interview",
    "Non-Music---Monolog",
    "Non-Music---Poetry",
    "Non-Music---Political",
    "Non-Music---Promotional",
    "Non-Music---Radioplay",
    "Non-Music---Religious",
    "Non-Music---Spoken Word",
    "Pop---Ballad",
    "Pop---Bollywood",
    "Pop---Bubblegum",
    "Pop---Chanson",
    "Pop---City Pop",
    "Pop---Europop",
    "Pop---Indie Pop",
    "Pop---J-pop",
    "Pop---K-pop",
    "Pop---Kayōkyoku",
    "Pop---Light Music",
    "Pop---Music Hall",
    "Pop---Novelty",
    "Pop---Parody",
    "Pop---Schlager",
    "Pop---Vocal",
    "Reggae---Calypso",
    "Reggae---Dancehall",
    "Reggae---Dub",
    "Reggae---Lovers Rock",
    "Reggae---Ragga",
    "Reggae---Reggae",
    "Reggae---Reggae-Pop",
    "Reggae---Rocksteady",
    "Reggae---Roots Reggae",
    "Reggae---Ska",
    "Reggae---Soca",
    "Rock---AOR",
    "Rock---Acid Rock",
    "Rock---Acoustic",
    "Rock---Alternative Rock",
    "Rock---Arena Rock",
    "Rock---Art Rock",
    "Rock---Atmospheric Black Metal",
    "Rock---Avantgarde",
    "Rock---Beat",
    "Rock---Black Metal",
    "Rock---Blues Rock",
    "Rock---Brit Pop",
    "Rock---Classic Rock",
    "Rock---Coldwave",
    "Rock---Country Rock",
    "Rock---Crust",
    "Rock---Death Metal",
    "Rock---Deathcore",
    "Rock---Deathrock",
    "Rock---Depressive Black Metal",
    "Rock---Doo Wop",
    "Rock---Doom Metal",
    "Rock---Dream Pop",
    "Rock---Emo",
    "Rock---Ethereal",
    "Rock---Experimental",
    "Rock---Folk Metal",
    "Rock---Folk Rock",
    "Rock---Funeral Doom Metal",
    "Rock---Funk Metal",
    "Rock---Garage Rock",
    "Rock---Glam",
    "Rock---Goregrind",
    "Rock---Goth Rock",
    "Rock---Gothic Metal",
    "Rock---Grindcore",
    "Rock---Grunge",
    "Rock---Hard Rock",
    "Rock---Hardcore",
    "Rock---Heavy Metal",
    "Rock---Indie Rock",
    "Rock---Industrial",
    "Rock---Krautrock",
    "Rock---Lo-Fi",
    "Rock---Lounge",
    "Rock---Math Rock",
    "Rock---Melodic Death Metal",
    "Rock---Melodic Hardcore",
    "Rock---Metalcore",
    "Rock---Mod",
    "Rock---Neofolk",
    "Rock---New Wave",
    "Rock---No Wave",
    "Rock---Noise",
    "Rock---Noisecore",
    "Rock---Nu Metal",
    "Rock---Oi",
    "Rock---Parody",
    "Rock---Pop Punk",
    "Rock---Pop Rock",
    "Rock---Pornogrind",
    "Rock---Post Rock",
    "Rock---Post-Hardcore",
    "Rock---Post-Metal",
    "Rock---Post-Punk",
    "Rock---Power Metal",
    "Rock---Power Pop",
    "Rock---Power Violence",
    "Rock---Prog Rock",
    "Rock---Progressive Metal",
    "Rock---Psychedelic Rock",
    "Rock---Psychobilly",
    "Rock---Pub Rock",
    "Rock---Punk",
    "Rock---Rock & Roll",
    "Rock---Rockabilly",
    "Rock---Shoegaze",
    "Rock---Ska",
    "Rock---Sludge Metal",
    "Rock---Soft Rock",
    "Rock---Southern Rock",
    "Rock---Space Rock",
    "Rock---Speed Metal",
    "Rock---Stoner Rock",
    "Rock---Surf",
    "Rock---Symphonic Rock",
    "Rock---Technical Death Metal",
    "Rock---Thrash",
    "Rock---Twist",
    "Rock---Viking Metal",
    "Rock---Yé-Yé",
    "Stage & Screen---Musical",
    "Stage & Screen---Score",
    "Stage & Screen---Soundtrack",
    "Stage & Screen---Theme",
]
mood_theme_classes = [
    "action",
    "adventure",
    "advertising",
    "background",
    "ballad",
    "calm",
    "children",
    "christmas",
    "commercial",
    "cool",
    "corporate",
    "dark",
    "deep",
    "documentary",
    "drama",
    "dramatic",
    "dream",
    "emotional",
    "energetic",
    "epic",
    "fast",
    "film",
    "fun",
    "funny",
    "game",
    "groovy",
    "happy",
    "heavy",
    "holiday",
    "hopeful",
    "inspiring",
    "love",
    "meditative",
    "melancholic",
    "melodic",
    "motivational",
    "movie",
    "nature",
    "party",
    "positive",
    "powerful",
    "relaxing",
    "retro",
    "romantic",
    "sad",
    "sexy",
    "slow",
    "soft",
    "soundscape",
    "space",
    "sport",
    "summer",
    "trailer",
    "travel",
    "upbeat",
    "uplifting"
]
instrument_classes = [
    "accordion",
    "acousticbassguitar",
    "acousticguitar",
    "bass",
    "beat",
    "bell",
    "bongo",
    "brass",
    "cello",
    "clarinet",
    "classicalguitar",
    "computer",
    "doublebass",
    "drummachine",
    "drums",
    "electricguitar",
    "electricpiano",
    "flute",
    "guitar",
    "harmonica",
    "harp",
    "horn",
    "keyboard",
    "oboe",
    "orchestra",
    "organ",
    "pad",
    "percussion",
    "piano",
    "pipeorgan",
    "rhodes",
    "sampler",
    "saxophone",
    "strings",
    "synthesizer",
    "trombone",
    "trumpet",
    "viola",
    "violin",
    "voice"
]

In [ ]:
!cd audiocraft

In [ ]:
from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D
import numpy as np

def filter_predictions(predictions, class_list, threshold=0.1):
    predictions_mean = np.mean(predictions, axis=0)
    sorted_indices = np.argsort(predictions_mean)[::-1]
    filtered_indices = [i for i in sorted_indices if predictions_mean[i] > threshold]
    filtered_labels = [class_list[i] for i in filtered_indices]
    filtered_values = [predictions_mean[i] for i in filtered_indices]
    return filtered_labels, filtered_values

def make_comma_separated_unique(tags):
    seen_tags = set()
    result = []
    for tag in ', '.join(tags).split(', '):
        if tag not in seen_tags:
            result.append(tag)
            seen_tags.add(tag)
    return ', '.join(result)

def get_audio_features(audio_filename):
    audio = MonoLoader(filename=audio_filename, sampleRate=16000, resampleQuality=4)()
    embedding_model = TensorflowPredictEffnetDiscogs(graphFilename="discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
    embeddings = embedding_model(audio)

    result_dict = {}

    # predict genres
    genre_model = TensorflowPredict2D(graphFilename="genre_discogs400-discogs-effnet-1.pb", input="serving_default_model_Placeholder", output="PartitionedCall:0")
    predictions = genre_model(embeddings)
    filtered_labels, _ = filter_predictions(predictions, genre_labels)
    filtered_labels = ', '.join(filtered_labels).replace("---", ", ").split(', ')
    result_dict['genres'] = make_comma_separated_unique(filtered_labels)

    # predict mood/theme
    mood_model = TensorflowPredict2D(graphFilename="mtg_jamendo_moodtheme-discogs-effnet-1.pb")
    predictions = mood_model(embeddings)
    filtered_labels, _ = filter_predictions(predictions, mood_theme_classes, threshold=0.05)
    result_dict['moods'] = make_comma_separated_unique(filtered_labels)

    # predict instruments
    instrument_model = TensorflowPredict2D(graphFilename="mtg_jamendo_instrument-discogs-effnet-1.pb")
    predictions = instrument_model(embeddings)
    filtered_labels, _ = filter_predictions(predictions, instrument_classes)
    result_dict['instruments'] = filtered_labels

    return result_dict

In [ ]:
#dataset_path = "/home/murali/data/original"

In [ ]:
!ls /home/murali/data/original

PANIVIZHUM_DM_TEMPO_85_4_4.wav	  SWAAMINAADHA_CM_TEMPO_102_4_4.wav
PRAMADAVANAM_CM_TEMPO_78_4_4.wav  VAARMUKILE_CM_TEMPO_80_4_4.wav


In [ ]:
# Create .jsonl from the extracted features, make a train/test split, and save in the right place.

# set the following variable to True if you want to see a progress bar instead of the printed results:
use_tqdm = False

import os
import json
import random
import librosa
from pydub import AudioSegment
import wave
import re

from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)

# make sure the .jsonl has a place to go
os.makedirs("/home/murali/audiocraft/egs/train", exist_ok=True)
os.makedirs("/home/murali/audiocraft/egs/eval", exist_ok=True)

train_len = 0
eval_len = 0
temp_dataset_path = dataset_path + '/original'
with open("/home/murali/audiocraft/egs/train/data.jsonl", "w") as train_file, \
     open("/home/murali/audiocraft/egs/eval/data.jsonl", "w") as eval_file:

    dset = tqdm(os.listdir(temp_dataset_path)) if use_tqdm else os.listdir(temp_dataset_path)
    print(dset)
    random.shuffle(dset)

    for filename in dset:
        result = get_audio_features(os.path.join(temp_dataset_path, filename))

        # TODO: make openai call, populate description and keywords

        # get key and BPM
        y, sr = librosa.load(os.path.join(temp_dataset_path, filename))
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        tempo = round(tempo) # not usually accurate lol
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        key = np.argmax(np.sum(chroma, axis=1))
        key = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'][key]
        length = librosa.get_duration(y=y, sr=sr)
        # print(f"{filename}: {result}, detected key {key}, detected bpm {tempo}")

        # THIS IS FOR MY OWN DATASET FORMAT
        # Meant strictly to extract from format: "artist name 4_chunk25.wav"
        # Modify for your own use!!
        def extract_artist_from_filename(filename):
            match = re.search(r'(.+?)\s\d+_chunk\d+\.wav', filename)
            artist = match.group(1) if match else ""
            return artist.replace("mix", "").strip() if "mix" in artist else artist
        artist_name = extract_artist_from_filename(filename)

        # populate json
        entry = {
            "key": f"{key}",
            "artist": artist_name,
            "sample_rate": 44100,
            "file_extension": "wav",
            "description": "",
            "keywords": "",
            "duration": length,
            "bpm": tempo,
            "genre": result.get('genres', ""),
            "title": "",
            "name": "",
            "instrument": result.get('instruments', ""),
            "moods": result.get('moods', []),
            "path": os.path.join(temp_dataset_path, filename),
        }
        print(entry)

        # train/test split
        if random.random() < 0.85:
            train_len += 1
            train_file.write(json.dumps(entry) + '\n')
        else:
            eval_len += 1
            eval_file.write(json.dumps(entry) + '\n')

print(train_len)
print(eval_len)

# clear cuda mem for finetuning
from numba import cuda
device = cuda.get_current_device()
device.reset()

['SWAAMINAADHA_CM_TEMPO_102_4_4.wav', 'PANIVIZHUM_DM_TEMPO_85_4_4.wav', 'MIZHIYAZHAK_DM_TEMPO_90_4_4.wav', 'ENTHITHRA_THAAMASAM_DM_TEMPO_90_4_4.wav', 'DASARADHA_CM_TEMPO_78_4_4.wav', 'AYYENGAARU_DM_TEMPO_100_4_4.wav', 'VAARMUKILE_CM_TEMPO_80_4_4.wav', 'PRAMADAVANAM_CM_TEMPO_78_4_4.wav']


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `discogs-effnet-bs64-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_discogs400-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_discogs400-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_moodtheme-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_moodtheme-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_instrument-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_instrument-discogs-effnet-1.pb`


{'key': 'D', 'artist': '', 'sample_rate': 44100, 'file_extension': 'wav', 'description': '', 'keywords': '', 'duration': 107.29410430839002, 'bpm': 112, 'genre': 'Classical, Contemporary, Neo-Classical, Post-Modern', 'title': '', 'name': '', 'instrument': ['piano'], 'moods': 'relaxing, emotional, film, background, melodic, romantic, drama, meditative, sad, melancholic, christmas', 'path': '/home/murali/data/samples/train_musicgen_edm_uncond/train/outputs/original/PANIVIZHUM_DM_TEMPO_85_4_4.wav'}


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `discogs-effnet-bs64-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_discogs400-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_discogs400-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_moodtheme-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_moodtheme-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_instrument-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_instrument-discogs-effnet-1.pb`


{'key': 'G', 'artist': '', 'sample_rate': 44100, 'file_extension': 'wav', 'description': '', 'keywords': '', 'duration': 110.4, 'bpm': 199, 'genre': 'Classical, Contemporary, Neo-Classical', 'title': '', 'name': '', 'instrument': ['piano'], 'moods': 'relaxing, background, emotional, film, drama, melodic, romantic, meditative, melancholic, sad', 'path': '/home/murali/data/samples/train_musicgen_edm_uncond/train/outputs/original/AYYENGAARU_DM_TEMPO_100_4_4.wav'}


...

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `discogs-effnet-bs64-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_discogs400-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_discogs400-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_moodtheme-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_moodtheme-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_instrument-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_instrument-discogs-effnet-1.pb`


{'key': 'C', 'artist': '', 'sample_rate': 44100, 'file_extension': 'wav', 'description': '', 'keywords': '', 'duration': 138.46154195011337, 'bpm': 152, 'genre': 'Classical, Contemporary, Electronic, Progressive Breaks, Trip Hop, Big Beat', 'title': '', 'name': '', 'instrument': ['piano'], 'moods': 'relaxing, background, emotional, film, melodic, romantic, melancholic, sad, drama, meditative, christmas, ballad', 'path': '/home/murali/data/samples/train_musicgen_edm_uncond/train/outputs/original/DASARADHA_CM_TEMPO_78_4_4.wav'}


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `discogs-effnet-bs64-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_discogs400-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `genre_discogs400-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_moodtheme-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_moodtheme-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_instrument-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_instrument-discogs-effnet-1.pb`


{'key': 'D', 'artist': '', 'sample_rate': 44100, 'file_extension': 'wav', 'description': '', 'keywords': '', 'duration': 77.33333333333333, 'bpm': 185, 'genre': 'Classical, Contemporary, Neo-Classical, Modern, Neo-Romantic', 'title': '', 'name': '', 'instrument': ['piano'], 'moods': 'relaxing, film, emotional, background, romantic, drama, melodic, meditative, christmas, melancholic, ballad', 'path': '/home/murali/data/samples/train_musicgen_edm_uncond/train/outputs/original/ENTHITHRA_THAAMASAM_DM_TEMPO_90_4_4.wav'}
8
0


CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

# load dataset into musicgen

a dataset for musicgen is:
- a .yaml file with basic info about the audio sample rate and channels, and a pointer to .jsonl files containing the prompt metadata and links to the corresponding audio files.
- the above .jsonl file
- a folder full of audio

it looks for the .yaml at `content/audiocraft/config/dset/audio/YOUR_TRAINING_RUN.yaml`, which you point to by setting `dset=audio/YOUR_TRAINING_RUN` in the `dora run` command. In my case, `YOUR_TRAINING_RUN` is `train` in the example code.

Put 15% of your dataset in `egs/eval` at random. To help mitigate overfitting.

- `train`: Data for training the model.
- `valid`: Data for hyperparameter tuning and early stopping during training.
- `evaluate`: Data for assessing the model's performance post-training.
- `generate`: Data used for output generation, often the same as `train` but not necessarily.

the `egs/YOUR_TRAINING_RUN` is referring to `content/audiocraft/egs/YOUR_TRAINING_RUN/data.jsonl`. the contents Eg:

```
{
  "key": "", "artist": "", "sample_rate": 44100, "file_extension": "wav",
  "description": "", "keywords": "", "duration": 30.0, "bpm": "", "genre": "",
  "title": "", "name": "", "instrument": "", "moods": [],
  "path": os.path.join(dataset_folder, filename),
}
```
any of these fields will be omitted if empty, only "path" is required.

## example:

To construct the dataset:
- all the audios are 30 seconds long @ 44100 samples
- they live in `content/drive/MyDrive/samples/train_musicgen_edm_uncond/train/outputs`
- the name of my training run ("YOUR_TRAINING_RUN") is "train"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# if there is a .json file for each audio file sharing the same filename, set this variable to True
use_existing_json = False
import os
import json
import random
import wave

# make sure the .jsonl has a place to go
os.makedirs("/home/audiocraft/egs/train", exist_ok=True)
os.makedirs("/home/audiocraft/egs/eval", exist_ok=True)

# dataset_folder = "/content/drive/MyDrive/samples/train_musicgen_edm_uncond/train/outputs"
dataset_folder = "/home/data/samples/train_musicgen_edm_uncond/train/outputs/original"
train_manifest_path = "/home/audiocraft/egs/train/data.jsonl"
eval_manifest_path = "/home/audiocraft/egs/eval/data.jsonl"

dataset_len = 0
train_len = 0
eval_len = 0

train_file = open(train_manifest_path, 'w')
eval_file = open(eval_manifest_path, 'w')

for filename in os.listdir(dataset_folder):
    if filename.endswith(".wav"):
        dataset_len += 1

        if use_existing_json:
            json_filepath = os.path.splitext(filename)[0] + ".json"
            if os.path.exists(json_filepath):
                with open(json_filepath, 'r') as json_file:
                    entry = json.load(json_file)
                    entry["path"] = os.path.join(dataset_folder, filename)
            else:
                print(f'error loading json: could not find {json_filepath}')
        else:
            import librosa
            y, sr = librosa.load(os.path.join(dataset_path, filename))
            length = librosa.get_duration(y=y, sr=sr)

            entry = {
                "key": "",
                "artist": "",
                "sample_rate": 44100,
                "file_extension": "wav",
                "description": "",
                "keywords": "",
                "duration": length,
                "bpm": "",
                "genre": "electronic",
                "title": "",
                "name": "",
                "instrument": "",
                "moods": [],
                "path": os.path.join(dataset_folder, filename),
            }

        if random.random() < 0.85:
            train_len += 1
            train_file.write(json.dumps(entry) + '\n')
        else:
            eval_len += 1
            eval_file.write(json.dumps(entry) + '\n')

train_file.close()
eval_file.close()

print(f'dataset length: {dataset_len} audio clips')
print(f'train length: {train_len} audio clips')
print(f'eval length: {eval_len} audio clips')

dataset length: 8 audio clips
train length: 5 audio clips
eval length: 3 audio clips


In [ ]:
# Make the .yaml

config_path = "/home/murali/audiocraft/config/dset/audio/train.yaml"

# point to the folders that your .jsonl is in
data_path = "egs/train"
eval_data_path = "egs/eval"

package = "package" # yay python not letting me put #@.package in a string :/
yaml_contents = f"""#@{package} __global__

datasource:
  max_channels: 2
  max_sample_rate: 44100

  evaluate: {eval_data_path}
  generate: {data_path}
  train: {data_path}
  valid: {eval_data_path}
"""

with open(config_path, 'w') as yaml_file:
    yaml_file.write(yaml_contents)

# Training

Musicgen - `Dora` for hyperparameter tuning and monitoring.

Start a finetuning run on the pretrained "small" model.
Meta's documentation here.](https://github.com/facebookresearch/audiocraft/blob/main/docs/MUSICGEN.md#training-stereo-models)

In [ ]:
import os
os.chdir('/home/audiocraft')

In [ ]:
%env USER=username
# !cd audiocraft

command = (
    "dora -P audiocraft run"
    " solver=musicgen/musicgen_base_32khz"
    " model/lm/model_scale=small"
    " continue_from=//pretrained/facebook/musicgen-small"
    " conditioner=text2music"
    " dset=audio/train"
    " dataset.num_workers=2"
    " dataset.valid.num_samples=1"
    " dataset.batch_size=2" # Variable
    " schedule.cosine.warmup=8"
    " optim.optimizer=adamw" # uses dadaw by default, which is worse for single-gpu runs
    " optim.lr=1e-4"
    " optim.epochs=5" # stops training after 5 epochs - variable
    " optim.updates_per_epoch=1000" # 2000 by default, vary this for checkpoints quicker ig
    " optim.adam.weight_decay=0.01"
    " generate.lm.prompted_samples=False" # skip super long generate step
    " generate.lm.gen_gt_samples=True"
)
!{command}
# print({command})

In [ ]:
# STEREO TRAINING EXAMPLE

%env USER=username

command = (
    "dora -P audiocraft run"
    " solver=musicgen/musicgen_stereo_finetune_32khz" # this config comes with all the stereo settings
    " model/lm/model_scale=small"
    " continue_from=//pretrained/facebook/musicgen-stereo-small" # point to the stereo model here
    " conditioner=text2music"
    " dset=audio/train"
    " dataset.num_workers=2"
    " dataset.valid.num_samples=1"
    " dataset.batch_size=2"
    " schedule.cosine.warmup=8"
    " optim.optimizer=adamw"
    " optim.lr=1e-4"
    " optim.epochs=5"
    " optim.updates_per_epoch=1000"
    " optim.adam.weight_decay=0.01"
    " generate.lm.prompted_samples=False" # skip super long generate step
    " generate.lm.gen_gt_samples=True"
)

# !{command}
print({command})

In [ ]:
# clear cuda mem
from numba import cuda
device = cuda.get_current_device()
device.reset()

# Save, load, export

In [ ]:
# Exporting .bin files from a training run:

from audiocraft.utils import export
from audiocraft import train

sig = "aec0903f"

# from https://github.com/facebookresearch/audiocraft/blob/main/docs/MUSICGEN.md#importing--exporting-models
xp = train.main.get_xp_from_sig(sig)
export.export_lm(xp.folder / 'checkpoint.th', '/content/checkpoints/finetune/state_dict.bin')
export.export_pretrained_compression_model('facebook/encodec_32khz', '/content/checkpoints/finetune/compression_state_dict.bin')


In [ ]:
# Loading a finetune for inference:

from audiocraft.models import MusicGen
musicgen = MusicGen.get_pretrained('/content/checkpoints/finetune')

In [ ]:
# Resuming a run:

sig = "aec0903f"

command = (
    "dora run solver=musicgen/musicgen_base_32khz"
    " model/lm/model_scale=small"

    f" continue_from=//SIG/{sig}"

    # save the .th file, load it in a new runtime, and resume:
    f" continue_from=/tmp/audiocraft_lyra/xps/{sig}/checkpoint.th"

    " conditioner=text2music"
    " dset=audio/train"
    " dataset.num_workers=2"
    " dataset.valid.num_samples=1"
    " dataset.batch_size=2"
    " schedule.cosine.warmup=8"
    " optim.optimizer=adamw" # uses dadaw by default, which is worse for single-gpu runs
    " optim.lr=1e-4"
    " optim.epochs=5" # stops training after 5 epochs- change this
    " optim.adam.weight_decay=0.01"
)

!{command}

In [ ]:
# Saving the .th file to google drive for persistence:
# Point directly to the checkpoint in google drive

import shutil
sig = "aec0903f"

source_path = f'/tmp/audiocraft_lyra/xps/{sig}/checkpoint.th'
destination_path = '/content/drive/MyDrive/musicgen_finetunes/checkpoints/new'
os.makedirs(destination_path, exist_ok=True)
shutil.copy(source_path, destination_path)

'/content/drive/MyDrive/musicgen_finetunes/checkpoints/new/checkpoint.th'

In [ ]:
# Attempt to save checkpoint on crash!!
sig = "aec0903f"

try:
    !{command}
except:
    import shutil
    source_path = f'/tmp/audiocraft_lyra/xps/{sig}/checkpoint.th'
    destination_path = '/content/drive/MyDrive/musicgen_finetunes/checkpoints/'
    os.makedirs(destination_path, exist_ok=True)
    shutil.copy(source_path, destination_path)

# generate

for more info on using the multi-band diffusion model, read the docs here:
https://github.com/facebookresearch/audiocraft/blob/main/docs/MBD.md

Note: To load the model from drive so you can use these examples without running the training script first, change `content/checkpoints/finetune` in the first cell to the path to the drive folder that the two `.bin` files are saved in. in the exporting example, it was `content/drive/MyDrive/musicgen_finetunes/checkpoints/`.

In [ ]:
from audiocraft.data.audio import audio_write
import IPython.display as ipd
from audiocraft.models import MusicGen
import numpy as np

# load your finetune
musicgen = MusicGen.get_pretrained('/content/checkpoints/finetune')
musicgen.set_generation_params(duration=16)

In [ ]:
# Example 1: unconditional generation

wavs = musicgen.generate_unconditional(4)

# save and display generated audio
for idx, one_wav in enumerate(wavs):
    audio_write(f'{idx}', one_wav.cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True)
    ipd.display(ipd.Audio(one_wav.cpu(), rate=32000))

In [ ]:
# Example 2: text guided generation

wavs = musicgen.generate([
    'disco',
    'slide guitar bluegrass',
    'breakbeat, amen break',
    'epic orchestral strings'
])

# save and display generated audio
for idx, one_wav in enumerate(wavs):
    audio_write(f'{idx}', one_wav.cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True)
    ipd.display(ipd.Audio(one_wav.cpu(), rate=32000))

In [ ]:
# helper functions for handling sample input and continations
# RUN THIS BEFORE RUNNING THE NEXT CELLS!

import julius, torch

def normalize_audio(audio_data):
    max_value = torch.max(torch.abs(audio_data))
    audio_data /= max_value
    return audio_data

def convert_audio_channels(wav: torch.Tensor, channels: int = 2) -> torch.Tensor:
    *shape, src_channels, length = wav.shape
    if src_channels == channels:
        pass
    elif channels == 1:
        wav = wav.mean(dim=-2, keepdim=True)
    elif src_channels == 1:
        wav = wav.expand(*shape, channels, length)
    elif src_channels >= channels:
        wav = wav[..., :channels, :]
    else:
        raise ValueError('The audio file has less channels than requested but is not mono.')
    return wav

def convert_audio(wav: torch.Tensor, from_rate: float, to_rate: float, to_channels: int) -> torch.Tensor:
    wav = julius.resample_frac(wav, int(from_rate), int(to_rate))
    wav = convert_audio_channels(wav, to_channels)
    return wav

# runs musicgen.generate_continuation in 30s chunks and appends them until it reaches generation_length

def generate_audio_continuation(musicgen, sample, generation_length, segment_length=30, overlap=10):
    overlap_samples = overlap * 32000
    segment_samples = segment_length * 32000
    output = np.array([])
    output = np.concatenate((output, sample.cpu().squeeze().numpy().astype(np.float32)))
    init_length = len(output) / 32000

    while len(output) / 32000 < generation_length:
        musicgen.set_generation_params(duration=segment_length)
        prompt = torch.tensor(np.array([output[-overlap_samples:]]), dtype=torch.float32)
        res = musicgen.generate_continuation(prompt=prompt, prompt_sample_rate=32000)
        res = res.cpu().squeeze().numpy().astype(np.float32)
        output = np.concatenate((output, res[overlap_samples:]))

    return output

In [ ]:
# Example 3: sample continuation
# run the helper functions cell or this won't work!

# upload your file
from google.colab import files
uploaded = files.upload()

input_audio_filename = next(iter(uploaded.keys()))
sample, sample_sr = torchaudio.load(input_audio_filename)
sample = normalize_audio(sample)
sample = convert_audio(sample, sample_sr, 32000, 1)

# generate
wav = generate_audio_continuation(musicgen, sample, 60)

# save and display generated audio
audio_write('continuation', output.cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True)
ipd.display(ipd.Audio(output, rate=32000))

In [ ]:
# Example 4: long generations (self-continuation)
# run the helper functions cell to make this work!

# this is unconditional for the example, swap in text guidance as needed.
wavs = musicgen.generate_unconditional(4)

# continuations only work on one sample at a time
for idx, wav in enumerate(wavs):

    wav = generate_audio_continuation(musicgen, wav, 60)

    audio_write(f'{idx}', wav.cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True)
    ipd.display(ipd.Audio(wav.cpu(), rate=32000))

In [ ]:
# Example 6: multiband diffusion decoder

from audiocraft.models import MusicGen, MultiBandDiffusion
mbd = MultiBandDiffusion.get_mbd_musicgen()

# use mbd to generate the audio from the codebook tokens
wavs, tokens = musicgen.generate_unconditional(4, return_tokens=True)
wavs_diffusion = mbd.tokens_to_wav(tokens)

# save and display generated audio
for idx, one_wav in enumerate(wavs):
    audio_write(f'{idx}', one_wav.cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True)
    audio_write(f'{idx}_diffusion', wavs_diffusion[idx].cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True)

    print('default decoder:')
    ipd.display(ipd.Audio(one_wav.cpu(), rate=32000))
    print('multiband diffusion:')
    ipd.display(ipd.Audio(wavs_diffusion[idx].cpu().cpu(), rate=32000))

In [ ]:
# Stereo inference
# Loading a custom checkpoint.


from audiocraft.models import MusicGen
import IPython.display as ipd

model = MusicGen.get_pretrained("facebook/musicgen-stereo-medium")
model.set_generation_params(duration=8)

wavs = model.generate([
    'disco',
    'slide guitar bluegrass',
    'breakbeat, amen break',
    'epic orchestral strings'
])

for idx, one_wav in enumerate(wavs):
    ipd.display(ipd.Audio(one_wav.cpu(), rate=model.sample_rate))


# local installation for inference (or training with good enough hardware)

## step 1 - environment

once conda's installed, run:

```
conda create --name musicgen
conda activate musicgen
```

make a folder for your musicgen projects to live. Use `cd` to this folder before running the commands in the next step.

## step 2 - audiocraft

Start installing.

Install Git

```
git clone https://github.com/facebookresearch/audiocraft.git
cd audiocraft
pip install -e .
pip install dora-search julius
```

running these out of the box should install dependencies, but it'll give a pytorch-cpu version with no GPU access by default. Need a CUDA version to actually run this.

## step 3 - torch with cuda

Need cuda for this to work: https://developer.nvidia.com/cuda-downloads

`pip install --upgrade torch --extra-index-url https://download.pytorch.org/whl/cu117`

You can't install torch with cuda *first*, because audiocraft will overwrite it during install.

## step 4 - make a script and run

Script to check:

```
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

print('loading model...')
model = MusicGen.get_pretrained('facebook/musicgen-small')
model.set_generation_params(duration=8)

print('generating...')
wav = model.generate_unconditional(4)

print('saving...')
for idx, one_wav in enumerate(wav):
    audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)

print('done!')
```

save this as `example.py` and run it toget a `1.wav` file.


# training from scratch with DAC

TBA, still being testing. Below code may help you get started but probably won't figure everything out for you:

```py
!git clone https://github.com/facebookresearch/audiocraft.git
%cd audiocraft
!pip install -e .
!pip install dora-search descript-audio-codec
!pip install -U protobuf
```

```py
%env USER=lyra
command = (
    "dora -P audiocraft run -d"
    " solver=musicgen/musicgen_base_32khz"
        # should probably put all this in a config, its no longer 32k
        # mostly just using this to handle defaults i dont want to worry about setting here
    " model/lm/model_scale=medium"
    " compression_model_checkpoint=//pretrained/dac_44khz"
    " sample_rate=44100"
    " transformer_lm.card=1024"
    " transformer_lm.n_q=9"
    " codebooks_pattern.modeling=delay"
    " codebooks_pattern.delay.delays=[0,1,2,3,4,5,6,7,8]"
    " conditioner=text2music"
    " dset=audio/train"
    " dataset.num_workers=2"
    " dataset.valid.num_samples=1"
    " dataset.batch_size=2"
    " schedule.cosine.warmup=8"
    " optim.optimizer=adamw"
    " optim.lr=1e-4"
    " optim.epochs=5"
    " optim.updates_per_epoch=10"
    " optim.adam.weight_decay=0.01"
    " fsdp.use=true"
    " autocast=false"
)
!{command}
```